In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import random
from keras.utils import to_categorical
import keras

In [ ]:
labels = pd.read_csv("/kaggle/input/dog-breed-identification/labels.csv")

In [ ]:
labels.head()

In [ ]:
### Importing all the images
x=[] ## variable to save all the images
z=[] ## varaible to save all breeds
from tqdm import tqdm
img_size=120

In [ ]:
for img,breed in tqdm(labels.values):
  images=cv2.imread("/kaggle/input/dog-breed-identification/train/{}.jpg".format(img),cv2.IMREAD_COLOR)
  images=cv2.resize(images,(img_size,img_size))
  x.append(images)
  z.append(breed)

In [ ]:
len(x)

In [ ]:
z

In [ ]:
len(x)

In [ ]:
len(z)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encdr=LabelEncoder()
y=encdr.fit_transform(z)


In [ ]:
### Chnaging the type of x
x=np.array(x)

In [ ]:
### Normalizing the x
x=x/255

In [ ]:
y

In [ ]:
#### spiltting the data into test and train
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0,stratify=y,shuffle=True)

In [ ]:
x_train.shape


In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
### viewing the image
index=7
y_train[index]
plt.imshow(x_train[index])

In [ ]:
index=8
y_train[index]
plt.imshow(x_train[index])

In [ ]:
index=9
y_train[index]
plt.imshow(x_train[index])

In [ ]:
index=15
y_train[index]
plt.imshow(x_train[index])

In [ ]:
index=20
y_train[index]
plt.imshow(x_train[index])

In [ ]:
index=25
y_train[index]
plt.imshow(x_train[index])

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
x_train

In [ ]:
x_test

In [ ]:

from keras.models import Sequential
from keras.layers import Dense,MaxPooling2D,Dropout,Flatten,Conv2D


In [ ]:
#### Model building
model=Sequential()
### Conv layer-1
model.add(Conv2D(32,(3,3),activation="relu",input_shape=(120,120,3)))
### Polling layer-1
model.add(MaxPooling2D((2,2)))
### Conv layer-2
model.add(Conv2D(32,(3,3),activation="relu"))
### Pooling layer-2
model.add(MaxPooling2D((2,2)))
### Flatten layer
model.add(Flatten())
### Hidden layer
model.add(Dense(250,activation="relu"))
### Output layer
model.add(Dense(120,activation="softmax"))


In [ ]:

## Compiling and fitting the model
model.compile(optimizer="Adam",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])
model.fit(x_train,y_train,epochs=5,batch_size=256,validation_split=0.2)

In [ ]:
### Checking the accuracy by increasing number of layers

#### Model building
model_2=Sequential()
### Conv layer-1
model_2.add(Conv2D(32,(3,3),activation="relu",input_shape=(120,120,3)))
### Polling layer-1
model_2.add(MaxPooling2D((2,2)))
### Conv layer-2
model_2.add(Conv2D(32,(3,3),activation="relu"))
### Pooling layer-2
model_2.add(MaxPooling2D((2,2)))
### Flatten layer
model_2.add(Flatten())
### Hidden layer
model_2.add(Dense(500,activation="relu"))
### Dropout -1
model_2.add(Dropout(0.5))
### Hidden layer -2
model_2.add(Dense(300,activation="relu"))
## Dropout -2
model_2.add(Dropout(0.5))
### hidden layer-3
model_2.add(Dense(250,activation="relu"))
### Output layer
model_2.add(Dense(120,activation="softmax"))

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
model_2

In [ ]:
model_tun

In [ ]:

## Compiling and fitting the model
model_2.compile(optimizer="Adam",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])
model_2.fit(x_train,y_train,epochs=5,batch_size=256,validation_split=0.2)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
model_2.evaluate(x_test,y_test)

In [ ]:
### if we increase the number of layers i am getting less accuarcy

In [ ]:
Tuning the dataset

In [ ]:
def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int("conv_1_filter",min_value=32,max_value=128,step=16),
                              kernel_size=hp.Choice("conv_1_kernel",values=[5,3]),
                              activation="relu",
                              input_shape=(28,28,1)
                          ),
                          keras.layers.Conv2D(
                              filters=hp.Int("conv_2_filter",min_value=32,max_value=64,step=16),
                              kernel_size=hp.Choice("conv_2_kernel",values=[5,3]),
                              activation="relu"
                          ),
                          keras.layers.Flatten(),
                          keras.layers.Dense(
                              units=hp.Int("dense_1_filter",min_value=32,max_value=128,step=16),
                              activation="relu"
                          ),
                          keras.layers.Dense(10,activation="softmax")
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate",values=[1e-2,1e-3])),
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
  return model

In [ ]:
tuner_search.search(x_train,y_train,epochs=3,validation_split=0.1)

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective="val_accuracy",
                          max_trials=5,
                          directory="output",
                          project_name="dog nad breed")

In [ ]:
model_tun=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model_tun.fit(x_train,y_train,epochs=10,initial_epoch=3,validation_split=0.1)